In [2]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch/minsearch.py

--2025-06-17 18:50:33--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5820 (5.7K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   5.68K  --.-KB/s    in 0s      

2025-06-17 18:50:33 (60.9 MB/s) - ‘minsearch.py.1’ saved [5820/5820]



In [18]:
import minsearch

In [19]:
import json

In [20]:
with open('documents.json', 'rt') as f_in:
    docs_raw =json.load(f_in)

In [21]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [22]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [23]:
index = minsearch.Index(
    text_fields=["question", "text", "selection"],
    keyword_fields=["course"]
)

In [24]:
q = "The course has already started, can I still enroll?"

In [25]:
index.fit(documents)

In [26]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [27]:
from openai import OpenAI
client = OpenAI()

In [28]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

In [30]:
response.choices[0].message.content

"Whether you can still enroll in a course that has already started depends on a variety of factors, including the institution's enrollment policies, the course structure, and the discretion of the instructor. Here are some steps you can take:\n\n1. **Contact the Instructor or Department**: Reach out to the course instructor or the department offering the course. They might allow late enrollment if you have a valid reason or if the course allows for flexible attendance.\n\n2. **Check Enrollment Policies**: Review the institution's policies on late enrollment. Some schools have a grace period for adding courses after the start date.\n\n3. **Consider the Course Format**: If it's an online or self-paced course, there might be more flexibility in joining after the start date.\n\n4. **Assess Your Ability to Catch Up**: Consider whether you'll be able to catch up on any missed content or assignments and if you’re prepared to put in the extra work to do so.\n\n5. **Administrative Approval**: Y

In [39]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: {question}

CONTEXT: {context}
""".strip()

In [42]:
context = ""
for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer:{doc['text']}\n\n"
prompt = prompt_template.format(question=q,context=context).strip()

In [44]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."